In [8]:
####################################################################################################################################################
#                                                               AGENTE PURA DIVERSÃO                                                               #
#--------------------------------------------------------------------------------------------------------------------------------------------------#
# Este agente tem como finalidade auxiliar qualquer pessoa a encontrar
# diversão no final de semana atual ou no próximo final de semana
# com base em algumas perguntas.
#--------------------------------------------------------------------------------------------------------------------------------------------------#
# Estas são algumas das perguntas a serem respondidas:
# 01) Em qual cidade e estado você estará?
# 02) Você tem alguma preferência de dia ou período para essas atividades? (Ex: sexta à noite, sábado durante o dia, domingo à tarde, ou está totalmente livre?)
# 03) Quem vai curtir esse momento com você? (Ex: vou sozinho(a), com meu amor, com a família e crianças – se sim, qual a faixa etária delas?, com um grupo de amigos?)
# 04) Para que tipo de programa você está mais inclinado(a)? Pense em categorias como: Gastronomia, Cultura, Ao Ar Livre, Esportes, Vida Noturna, Bem-Estar e Relaxamento,
#     Compras ou Algo totalmente diferente ou inusitado que você sempre quis experimentar?
# 05) Qual a 'vibe' que você procura para este final de semana? Algo mais agitado e social, super tranquilo e relaxante, culturalmente enriquecedor, cheio de aventura, romântico, ou talvez uma mistura?
# 06) Existe algum tipo de programa ou atividade que você gostaria de evitar a todo custo neste final de semana?
# 07) Falando de investimento, você tem uma faixa de orçamento em mente para essas atividades? (Ex: busco opções gratuitas, algo mais em conta até R$ X por pessoa, posso gastar um pouco mais, ou o céu é o limite?)
# 08) Até que distância você toparia se deslocar a partir do seu ponto principal (casa/hotel) para encontrar a atividade ideal?
# 09) Você já tem alguma ideia guardada na manga, algum lugar que sempre quis conhecer ou algo que viu e ficou curioso(a)?
# 10) Lembrando de outros finais de semana, teve alguma experiência de lazer que você AMOU ou alguma que DETESTOU? Compartilhar isso me ajuda a entender ainda mais o seu perfil!

%pip -q install google-genai

In [9]:
# Configura a API Key do Google Gemini

# Lib Python para acesso a recursos do sistema operacional
import os

# Lib Google Colab para acesso a dados do usuário
from google.colab import userdata

# Configurando uma variável de ambiente
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [10]:
# Configura o cliente da SDK do Gemini

# Importando a lib Google Gemini
from google import genai

# Instanciando um client do Gemini
client = genai.Client()

# Definindo o modelo a ser utilizado
MODEL_ID = "gemini-2.0-flash"

In [11]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk   # o parâmetro -q oculta aqueles vários detalhes da instalação

In [25]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search  # outras tools: maps, viagens, etc...

import os
from datetime import datetime
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown, HTML # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [13]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente (orquestrador)
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [14]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [15]:
######################################################
# --- Obtendo o contexto
######################################################
def obter_contexto():
    localizacao = ""
    periodo = ""
    acompanhantes = ""
    idade = "0"

    print(Markdown(f"***********   IMPORTANTE   ***********\nPara encerrar este agente a qualquer momento, digite no campo de resposta **fim**\n\n\n"))

    while localizacao != "fim":
        localizacao = input("Pergunta 01 de 10 ❓ Me informe a sua lozalização (cidade e estado): ")
        if not localizacao:
            print("\nSaber sua localização é muito importante para eu buscar atividade que sejam relevantes para você...\n\n")
        else:
            if localizacao == "fim":
                return localizacao, periodo, acompanhantes, idade
            else:
                print(f"\nÓtimo! Então você procura alguma atividade em {localizacao}...\n")
                break

    while periodo != "fim":
        periodo = input("Pergunta 02 de 10 ❓ Agora me diga em qual período você tem maior interesse (Ex.: manhã,\n tarde, noite, sábado a tarde, domingo em qualquer horário, etc.): ")
        if not periodo:
            print("\nVocê não me informou um período, vou assumir que posso buscar em qualquer dia ou horário, ok?\n\n")
            periodo = "Qualquer dia e horário" # Valor default
            break
        else:
            if periodo == "fim":
                return localizacao, periodo, acompanhantes, idade
            else:
                print(f"\nEntendi! Prefere que seja {periodo}\n")
                break


    while acompanhantes != "fim":
        acompanhantes = input("Pergunta 03 de 10 ❓ Quem irá com você (ex.: vou sozinho(a), com um grupo de amigos, com meu amor,\n com a família e crianças – se sim, qual a faixa etária delas?): ")

        if acompanhantes == "":
            print("\nVocê não me informou quem irá com você, vou assumir que você irá sozinho(a), ok?\n\n")
        else:
            if acompanhantes == "fim":
                return localizacao, periodo, acompanhantes, idade
            else:
                if acompanhantes.find("sozinh",1) > 0:
                    quem = "você"
                else:
                    quem = "vocês"
                print(f"\nExcelente! Vou procurar ótimas opções em {localizacao} para {quem}\n" )
                break


    while idade != "fim":
        idade = input("Pergunta 04 de 10 ❓ Me informe por favor a sua idade: ")
        if idade == "0":
            print("\nVocê não me informou sua idade, essa informação me ajudará muito na busca de atividades interessantes para você...\n\n")
        else:
            if idade == "fim":
                return localizacao, periodo, acompanhantes, idade
            else:
                if acompanhantes.find("sozinh",1) > 0:
                    quem = "você"
                else:
                    quem = "vocês"
                print(f"\n\nObrigado por essas primeiras informações! Agora vou querer saber um pouco mais dos seus interesses...\n\n" )
                break

    #return 'Localização: \n   - ' + localizacao + '\nPeríodo: \n   - ' + periodo + '\nAcompanhantes:   - ' + acompanhantes + '\nIdade: \n   - ' + idade
    return localizacao, periodo, acompanhantes, idade


In [16]:


######################################################
# --- Obtendo os gostos e o "mood"
######################################################

def obter_gostos():
    interesse = ""
    vibe = ""
    evitar = ""

    while interesse != "fim":
        interesse = input("Pergunta 05 de 10 ❓ Para que tipo de programa você está mais inclinado(a) (ex.: gastronomia, cultura, ao ar livre, esportes, \n" +
                          "vida noturna, bem-estar e relaxamento, compras ou algo totalmente diferente ou inusitado que você sempre quis experimentar?): ")
        if not interesse or interesse == "":
            print("\nO seu interesse é importantíssimo para eu buscar atividade que sejam relevantes para você...\n\n")
        else:
            if interesse == "fim":
                return interesse, vibe, evitar
            else:
                print(f"\nÓtima escolha! Vou seguir observando seu interesse em {interesse}...\n")
                break


    while vibe != "fim":
        vibe = input("Pergunta 06 de 10 ❓ Qual a 'vibe' que você procura para este final de semana? Algo mais agitado e social, super tranquilo\n" +
                     "e relaxante, culturalmente enriquecedor, cheio de aventura, romântico, ou talvez uma mistura?: ")
        if not vibe or vibe == "":
            print("\nO seu interesse é importantíssimo para eu buscar atividade que sejam relevantes para você...\n\n")
        else:
            if vibe == "fim":
                return interesse, vibe, evitar
            else:
                print(f"\nEntendi, já estou imaginando nessa vibe {vibe}...\n")
                break


    while evitar != "fim":
        evitar = input("Pergunta 07 de 10 ❓ Existe algum tipo de programa ou atividade que você gostaria de *evitar* a todo custo?: ")

        if not evitar or evitar == "":
            print("\Entendendo que você não tem nenhuma restrição e seguirei assim, ok?\n\n")
            evitar = "Não tenho nenhuma restrição"
            break
        else:
            if evitar == "fim":
                return interesse, vibe, evitar
            else:
                print(f"\nPode deixar que vou evitar {evitar}...\n")
                break

    print(f"\n\nChegamos num ponto muito importante, agora vamos falar de detalhes práticos para a escolha\n")

    #return 'Interesse: \n   - ' + interesse + '\nVibe: \n   - ' + vibe + '\nEvitar:   - ' + evitar
    return interesse, vibe, evitar


In [17]:

######################################################
# --- Obtendo detalhes práticos
######################################################

def obter_detalhes():
    investimento = ""
    distancia = ""
    curiosidade = ""

    while investimento != "fim":
        investimento = input("Pergunta 08 de 10 ❓ Falando de investimento, você tem uma faixa de orçamento em mente para essas atividades? (ex: busco opções gratuitas,\n" +
                             "algo mais em conta até R$ X por pessoa, posso gastar um pouco mais, ou o céu é o limite?): ")
        if not investimento:
            print("\Xiiiii... Me ajuda aí vai? Esse é um parâmetro importante para minhas decisões do que lhe oferecer de opção.\n\n")
        else:
            if investimento == "fim":
                return investimento, distancia, curiosidade
            else:
                print(f"\nMaravilha! Não vou esquecer de levar isso em consideração...\n")
                break


    while distancia != "fim":
        distancia = input("Pergunta 09 de 10 ❓ Até que distância você toparia se deslocar a partir do seu ponto principal (casa/hotel) para encontrar a atividade ideal?: ")
        if not distancia:
            print("\nVou considerar que só deseja algo na própria cidade, ok?.\n\n")
            distancia = "Somente aqui na cidade"
            break
        else:
            if distancia == "fim":
                return investimento, distancia, curiosidade
            else:
                print(f"\nVamos ver o que eu encontrarei nessa distância...\n")
                break


    while curiosidade != "fim":
        curiosidade = input("Pergunta 10 de 10 ❓ Você já tem alguma ideia guardada na manga, algum lugar que sempre quis conhecer ou algo que viu e ficou curioso(a)?: ")
        if not curiosidade:
            print("\nVou considerar que só deseja algo na própria cidade, ok?.\n\n")
            curiosidade = "Não tenho nenhuma ideia"
            break
        else:
            if curiosidade == "fim":
                return investimento, distancia, curiosidade
            else:
                print(f"\nIsso vai me ajudar a identificar melhor algusn dos seus interesses...\n")
                break


    #return 'Investimento: \n   - ' + investimento + '\nDistância Máxima: \n   - ' + distancia + '\nTenho Curiosidade Por:   - ' + curiosidade
    return investimento, distancia, curiosidade

In [18]:
################################################
# --- Agente: Buscador de Entretenimento   --- #
################################################
def agente_buscador(data_de_hoje, infos_do_cliente):
    instrucao = f"""Você é um Assistente de Entretenimento.
        Seu objetivo é montar um roteiro de programas e atividades
        para a próxima semana a partir da data de hoje utilizando ferramenta de busca do Google (google_search)
        e também sua base de dados local, levando em conta os seguintes parâmetros do cliente:
            1. Localização atual:
                - Cidade, bairro ou região de onde o cliente partirá
            2. Período(s) do dia de interesse:
                - (ex.: manhã, tarde, noite, qualquer horário)
            3. Acompanhantes:
                - Quem irá junto (ex.: família, amigos, crianças) e idades de cada pessoa.
            4. Tipos de programas/atividades de interesse:
                - (ex.: passeios ao ar livre, exposições, shows, aulas, tours guiados).
            5. Categorias preferidas:
                - (ex.: gastronomia, cultura, esportes, bem-estar, vida noturna).
            6. Atividades a evitar:
                - (ex.: lugares muito barulhentos, ambientes fechados, trilhas difíceis).
            7. Orçamento por pessoa:
                - Valor máximo estimado (ex.: até R$ 100, R$ 200–300).
            8. Distância máxima de deslocamento:
                - Raio de viagem a partir da localização (em km ou tempo de deslocamento).
            9. Pontos de curiosidade:
                - Lugares específicos que o cliente já demonstrou interesse em conhecer.

        **Instruções de Execução:**
        - Para cada dia da próxima semana, sugira **2 a 5 opções** de programa, cobrindo o(s) período(s) de interesse.
        - Considere nas opções selecionadas o entusiasmo das notícias sobre ele.
        - Se um dos lançamentos encontrados tiver poucas notícias ou poucas reações entusiasmadas, é possível que ele não seja tão relevante e pode ser substituído por outro que tenha mais.
        - Informe título do programa, breve descrição (1–2 frases), custo aproximado por pessoa, endereço/localização e link oficial ou site para reserva/ingressos.
        - Caso haja opção gratuita, destaque-a como “Gratuito”.
        - Se alguma atividade exigir reserva antecipada, mencione o prazo mínimo.
        - Verifique disponibilidade ou horários atualizados (datas e horários exatos).
        - Agrupe as sugestões por dia e período, por exemplo:

            **Segunda-feira (Manhã)**
            1. Passeio no Parque X – descrição…
            2. Oficina de Cerâmica no Ateliê Y – descrição…

            **Segunda-feira (Tarde)**
            1. …

        - Ao final, inclua um breve comparativo entre as opções (prós e contras) para ajudar o cliente na escolha.

        **Exemplo de Uso**:

        Informações do cliente:
        - Localização: São Paulo, Vila Madalena
        - Período: Tarde e noite
        - Acompanhantes: Casal (30 e 28 anos)
        - Interesses: Gastronomia, cultura, vida noturna
        - Evitar: Baladas com som muito alto
        - Orçamento: até R$ 200 por pessoa
        - Distância: até 10 km
        - Curiosidade: Quer conhecer o novo bar temático Z

        *Gere um roteiro de segunda-feira a domingo com base nesses dados.*"""


    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Buscador #################################################
        tools=[google_search],
        description="Agente que busca opções de programas ou atividades de acordo com o perfil informado pelo cliente",
        instruction=instrucao
    )

    entrada_do_agente_buscador = f"Data de hoje: {data_de_hoje}\nInformações do cliente: {infos_do_cliente}"

    # Executa o agente
    atividades_encontradas = call_agent(buscador, entrada_do_agente_buscador)
    return atividades_encontradas


In [19]:
################################################
# --- Agente: Diagramador                  --- #
################################################
def agente_diagramador(conteudo):
    redator = Agent(
        name="agente_diagramador",
        #model="gemini-2.5-pro-preview-03-25",
        #model = "gemini-2.0-pro-exp",
        #model = "gemini-1.5-pro-latest",
        model="gemini-2.0-flash",
        tools=[google_search],

        instruction="""
            Você é um Editor e diagramador experiente e criativo, com grande conhecimento em escrita de código HTML.
            Você é acostumado a escrever e diagramar muitos conteúdos de entretenimento para os mais diversos públicos.
            Utilize o conteudo fornecido, reorganize e enriqueça-os com citações e links de reportagens que você pode obter
            através de pesquisa no Google Search (google_search), para criar uma página HTML atrativa e elegante, com uma
            paleta de cores em tons pastéis que divulgará as sugestões de diversão para o público.
            """,
        description="Agente diagramador de conteúdos em formato HTML"
    )
    entrada_do_agente_redator = f"Conteúdo: {conteudo}"
    # Executa o agente
    texto = call_agent(redator, entrada_do_agente_redator)
    return texto

In [60]:
# Lista os modelos de IA disponíveis
client = genai.Client()
for model in client.models.list():
  print(model)


name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=1024 output_token_limit=1 supported_actions=['embedText', 'countTextTokens'] default_checkpoint_id=None checkpoints=None
name='models/gemini-1.0-pro-vision-latest' display_name='Gemini 1.0 Pro Vision' description='The original Gemini 1.0 Pro Vision model version which was optimized for image understanding. Gemini 1.0 Pro Vision was deprecated on July 12, 2024. Move to a newer Gemini version.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=12288 output_token_limit=4096 supported_actions=['generateContent', 'countTokens'] default_checkpoint_id=None checkpoints=None
name='models/gemini-pro-vision' display_name='Gemini 1.0 Pro Vi

In [29]:
def executa_agentes():
    data_de_hoje = date.today().strftime("%d/%m/%Y")

    print("\n\n*******🚀 Iniciando o Agente Pura Diversão!!! 🚀*******\n\n\n")

    (localizacao, periodo, acompanhantes, idade) = obter_contexto()
    if localizacao == "fim" or periodo == "fim" or acompanhantes == "fim" or idade == "fim":
        return "fim"

    (interesse, vibe, evitar) = obter_gostos()
    if interesse == "fim" or vibe == "fim" or evitar == "fim":
        return "fim"

    (investimento, distancia, curiosidade) = obter_detalhes()

    if investimento == "fim" or distancia == "fim" or curiosidade == "fim":
        return "fim"

    print("\n\n\n******************************************* Iniciando a busca das melhores opçções para você... *************************************************\n\n\n")
    infos_do_cliente = f"""- Localização: {localizacao}\n- Período: {periodo}\n- Acompanhantes: {acompanhantes} e eu tenho {idade}\n
                   - Interesses: {interesse}, com uma vibe {vibe}\n - Evitar: {evitar}\n- Orçamento: {investimento}\n
                   - Distância: {distancia}\n- Curiosidades: {curiosidade}"""

    resultado = agente_buscador(data_de_hoje, infos_do_cliente)

    return resultado


In [23]:
def apresenta_resultado(resultado):
    texto = agente_diagramador(resultado)
    print("\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n")

    #print("\n************************************************************ Tá aqui as nossas recomendações para você ************************************************************\n")
    display(HTML(texto))



In [33]:
resultado = executa_agentes()

if resultado != "fim":
    apresenta_resultado(resultado)
else:
    display(HTML("Até a próxima..."))






*******🚀 Iniciando o Agente Pura Diversão!!! 🚀*******



<IPython.core.display.Markdown object>
Pergunta 01 de 10 ❓ Me informe a sua lozalização (cidade e estado): São Paulo/SP

Ótimo! Então você procura alguma atividade em São Paulo/SP...

Pergunta 02 de 10 ❓ Agora me diga em qual período você tem maior interesse (Ex.: manhã,
 tarde, noite, sábado a tarde, domingo em qualquer horário, etc.): quarta-feira a tarde

Entendi! Prefere que seja quarta-feira a tarde

Pergunta 03 de 10 ❓ Quem irá com você (ex.: vou sozinho(a), com um grupo de amigos, com meu amor,
 com a família e crianças – se sim, qual a faixa etária delas?): grupo de amigos de 10 a 15 anos

Excelente! Vou procurar ótimas opções em São Paulo/SP para vocês

Pergunta 04 de 10 ❓ Me informe por favor a sua idade: 10 anos


Obrigado por essas primeiras informações! Agora vou querer saber um pouco mais dos seus interesses...


Pergunta 05 de 10 ❓ Para que tipo de programa você está mais inclinado(a) (ex.: gastronomia, cultura, 